In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from functools import reduce
%matplotlib inline

Merging Waves

In [ ]:
wave1 = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0001\21600-0001-Data.tsv", sep = '\t', usecols = ['AID'])
wave3educ = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0016\21600-0016-Data.tsv", sep = '\t', usecols = ['AID'])
wave3grad = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0017\21600-0017-Data.tsv", sep = '\t', usecols = ['AID'])
wave4 = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0022\21600-0022-Data.tsv", sep = '\t', usecols = ['AID'])

In [ ]:
waves = [wave1, wave3educ, wave3grad, wave4]
AID = reduce(lambda left, right: pd.merge(left, right, on = 'AID', how = 'inner'), waves)
len(AID)

GPA

In [ ]:
hs_gpa = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0016\21600-0016-Data.tsv", sep = '\t', usecols = ['AID', 'EAOGPAC'])
hs_gpa = hs_gpa.rename(columns = {'AID': 'AID', 'EAOGPAC': 'cumulative_gpa'})
hs_gpa = pd.merge(AID, hs_gpa, on = 'AID', how = 'left')
#hs_gpa.isna().sum() #check for nulls to make sure the AID matched
bins = [-1, 1, 2, 3 ,4]
labels = ['0-1', '1-2', '2-3', '3-4']
hs_gpa['gpa_range'] = pd.cut(np.array(hs_gpa['cumulative_gpa']), bins = bins, labels = labels)
hs_gpa = hs_gpa.drop(hs_gpa[hs_gpa['cumulative_gpa'] == 9995].index)
hs_gpa

Grades -- Wave 1

In [ ]:
grades = pd.read_csv(r"\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0001\21600-0001-Data.tsv", sep = '\t', usecols = ['AID','H1ED11', 'H1ED12', 'H1ED13', 'H1ED14'], low_memory = False)
grades = grades.rename(columns = {'AID': 'AID','H1ED11': 'English', 'H1ED12' : 'Math', 'H1ED13' : 'History', 'H1ED14' : 'Science'})
grades

In [ ]:
grades[['English', 'Math', 'History', 'Science']].corr()

Academic Attainment

In [ ]:
highest_educ = pd.read_excel(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\Wave 4 Questionaire .xlsx", 'highest_educ_level')
highest_educ = highest_educ.rename(columns = highest_educ.iloc[0])
highest_educ = highest_educ.drop(highest_educ.index[0])
highest_educ = highest_educ[['Value', 'Label']]
highest_educ

In [ ]:
value = [1,2,3,4,5,6,7,8,9,10,11,12,13,98]
educ = ['8th grade or less', 'some high school', 'high school graduate', 'some vocational/technical training', 'completed vocational/technical training', 'some college', "completed college (bachelor's degree)", 'some graduate school', "completed a master's degree", "some graduate training beyond a master's degree", 'completed a doctoral degree', 'some post baccalaureate professional education', 'completed post baccalaureate professional education', "don't know"]

educ_code = pd.DataFrame(list(zip(value, educ)), columns = ['Value', 'Highest_educ_level'])

In [ ]:
high_educ = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0022\21600-0022-Data.tsv", sep = '\t', usecols = ['AID', 'H4ED2'])
high_educ = high_educ.rename(columns = {'H4ED2':'Value'})
high_educ

In [ ]:
#high_educ['Value'].value_counts()

In [ ]:
educ_level = pd.merge(high_educ, educ_code, on='Value', how='left')
educ_level

In [ ]:
educ_level.isna().sum()

Peers

In [ ]:
peers = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0001\21600-0001-Data.tsv", sep = '\t', usecols = ['AID','H1ED18','H1ED19', 'H1ED24'])
peers = peers.rename(columns = {'H1ED18':'getting_along','H1ED19':'close_peers','H1ED24':'safe_school'})
peers = pd.merge(AID, peers, how = 'left', on = 'AID')
#peers.isna().sum()
#peers['getting_along'].value_counts() # get rid of 7 (57 ppl)
#peers['close_peers'].value_counts() #get rid of 7 (57 ppl) and 8 (1)
#len(peers.loc[peers['getting_along'] == 7])#= 57, same 57 state 7
#peers.loc[peers['close_peers'] == 8] # keep this value b/c they have answers for the other columns
peers = peers.drop(peers[peers['getting_along'] == 7].index)
peers

In [ ]:
code_ga = [0,1,2,3,4,6,7,8]
value_ga = ['never', 'just a few times', 'about once a week', 'almost everyday', 'everyday', 'refused', 'legitimate skip', "don’t know"]
ga_code = pd.DataFrame(list(zip(code_ga, value_ga)), columns = ['getting_along', 'ga_code'])

In [ ]:
peers = pd.merge(peers, ga_code, how = 'left', on = 'getting_along')

In [ ]:
code_agree = [1,2,3,4,5,6,7,8]
value_agree = ['strongly agree', 'agree', 'neither agree nor disagree', 'disagree', 'strongly disagree', 'refused', 'legitimate skip', "don't know"]

cp_code = pd.DataFrame(list(zip(code_agree, value_agree)), columns = ['close_peers', 'close_peers_code'])
cp_code

ss_code = pd.DataFrame(list(zip(code_agree, value_agree)), columns = ['safe_school', 'safe_school_code'])
ss_code

In [ ]:
peers = pd.merge(peers, cp_code, on = 'close_peers', how = 'left')
peers

In [ ]:
peers = pd.merge(peers, ss_code, on = 'safe_school', how = 'left')
peers

In [ ]:
peers = pd.merge(peers, hs_gpa, on = 'AID', how = 'inner')
len(peers)

In [ ]:
peers[['getting_along', 'close_peers', 'safe_school', 'cumulative_gpa']].corr()

In [ ]:
peer_grades = pd.merge(peers, grades, on = 'AID', how = 'inner')
peer_grades

In [ ]:
peer_grades[['getting_along', 'close_peers', 'safe_school','English', 'Math', 'History', 'Science']].corr()

In [ ]:
peer_grades[['English', 'Math', 'History', 'Science', 'cumulative_gpa']].corr()

In [ ]:
peer_educ_level = pd.merge(peer_grades, educ_level, how = 'left', on = 'AID')
peer_educ_level['Value'].value_counts()
len(peer_educ_level)

In [ ]:
peer_educ_level[['getting_along','close_peers', 'safe_school', 'cumulative_gpa', 'Value']].corr()

In [ ]:
peer_educ_level.head()

In [ ]:
peer_educ_level.isna().sum()

In [ ]:
peer_educ_level.to_csv('peers.csv')

Friends

In [ ]:
friends = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0001\21600-0001-Data.tsv", sep = '\t', usecols = ['AID','H1PR4'])
friends = friends.rename(columns = {'H1PR4': 'caring_friends'})
friends = pd.merge(AID, friends, how = 'left', on = 'AID')
#friends.isna().sum()
#friends['caring_friends'].value_counts() #drop 6, 96, 98
friends = friends.drop(friends[(friends['caring_friends'] == 6) |(friends['caring_friends'] == 96) | (friends['caring_friends'] == 98)].index)
friends

In [ ]:
friends = pd.merge(friends, hs_gpa, on = 'AID', how = 'inner')
len(friends)

In [ ]:
friends[['caring_friends', 'cumulative_gpa']].corr()

In [ ]:
friends_grades = pd.merge(friends, grades, on = 'AID', how = 'inner')
friends_grades

In [ ]:
friends_grades[['caring_friends', 'English', 'Math', 'History', 'Science']].corr()

In [ ]:
friends_grades[['cumulative_gpa', 'English', 'Math', 'History', 'Science']].corr()

In [ ]:
friends_educ_level = pd.merge(friends_grades, educ_level, how = 'left', on = 'AID')
friends_educ_level['Value'].value_counts()
len(friends_educ_level)

In [ ]:
friends_educ_level[['caring_friends', 'cumulative_gpa', 'Value']].corr()

In [ ]:
friends_educ_level.plot(kind = 'scatter',
                       x = 'caring_friends',
                       y = 'cumulative_gpa',
                       figsize = (10,6))

In [ ]:
friends_educ_level.to_csv('friends.csv')

In [ ]:
friends_educ_level

In [ ]:
code = [1,2,3,4,5,6,96,98]
value = ['not at all', 'very little', 'somewhat', 'quite a bit', 'very much', 'does not apply', 'refused', "don’t know"]

code_cf = pd.DataFrame(list(zip(code, value)), columns = ['caring_friends','caring_friends_code'])

In [ ]:
friends_code = pd.merge(friends_educ_level, code_cf, how = 'left', on = 'caring_friends')
friends_code

In [ ]:
friends_code.to_csv('friends.csv')

Mom and Dad Code

In [ ]:
code_rel = [1,2,3,4,5,6,7,8]
value_rel = ['not at all', 'very little', 'somewhat', 'quite a bit', 'very much', 'refused', 'legitimate skip', "don't know"]

Mom -- Relationship

In [ ]:
mom = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0001\21600-0001-Data.tsv", sep = '\t', usecols = ['AID','H1WP9', 'H1WP10'])
mom = mom.rename(columns = {'H1WP9': 'mom_close', 'H1WP10': 'caring_mom'})
mom = pd.merge(AID, mom, how = 'left', on = 'AID')
mom.isna().sum()
mom['mom_close'].value_counts() #drop 8 and 6 (2 each) and 7 (148)
mom['caring_mom'].value_counts() #drop 8 and 6 if overlap (1 each) and 7 (148)
mom.loc[(mom['mom_close'] == 6) | (mom['mom_close'] == 8)]
mom = mom.drop(mom[mom['mom_close'] == 7].index)
mom = mom.drop([1097, 3054])
mom

In [ ]:
mc_code = pd.DataFrame(list(zip(code_rel, value_rel)), columns = ['mom_close', 'mom_close_code'])
cm_code = pd.DataFrame(list(zip(code_rel, value_rel)), columns = ['caring_mom', 'caring_mom_code'])

In [ ]:
mom = pd.merge(mom, mc_code, how = 'left', on = 'mom_close')

In [ ]:
mom = pd.merge(mom, cm_code, how = 'left', on = 'caring_mom')

In [ ]:
mom = pd.merge(mom, hs_gpa, how = 'inner', on = 'AID')
mom

In [ ]:
mom[['mom_close', 'caring_mom', 'cumulative_gpa']].corr()

In [ ]:
mom_grades = pd.merge(mom, grades, on = 'AID', how = 'inner')
mom_grades

In [ ]:
mom_grades[['mom_close', 'caring_mom', 'English', 'Math', 'History', 'Science']].corr()

In [ ]:
mom_grades[['cumulative_gpa', 'English', 'Math', 'History', 'Science']].corr()

In [ ]:
mom_educ_level = pd.merge(mom_grades, educ_level, how = 'left', on = 'AID')
mom_educ_level['Value'].value_counts()
mom_educ_level.head()

In [ ]:
mom_educ_level[['mom_close', 'caring_mom', 'cumulative_gpa', 'Value']].corr()

In [ ]:
mom_educ_level['gpa_range'].value_counts()

In [ ]:
mom_educ_level.to_csv('mom.csv')

Mom -- View on School -- Disappointed if they don't graduate college/hs 

In [ ]:
mom_school = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0001\21600-0001-Data.tsv", sep = '\t', usecols = ['AID', 'H1WP11', 'H1WP12'])
mom_school = mom_school.rename(columns = {'H1WP11': 'graduate_college', 'H1WP12': 'graduate_hs'})
mom_school = pd.merge(AID, mom_school, on = 'AID', how = 'left')
mom_school.isna().sum()
mom_school['graduate_college'].value_counts() #drop 6(1) and 8(7) and 7 (148)
mom_school['graduate_hs'].value_counts() #drop 6 (2), 8 (7), 9(1) that overlap and 7 (148)
mom_school.loc[(mom_school['graduate_hs'] == 6) | (mom_school['graduate_hs'] == 8) | (mom_school['graduate_hs'] == 9)]
mom_school = mom_school.drop([1097, 2736, 2892, 3054])
mom_school = mom_school.drop(mom_school[mom_school['graduate_hs'] == 7].index)
mom_school

In [ ]:
mom_school = pd.merge(mom_school, hs_gpa, on = 'AID', how = 'inner')
mom_school

In [ ]:
mom_school[['graduate_college', 'graduate_hs', 'cumulative_gpa']].corr()

In [ ]:
mom_school_grades = pd.merge(mom_school, grades, on = 'AID', how = 'inner')
mom_school_grades

In [ ]:
mom_school_grades[['graduate_college', 'graduate_hs','English', 'Math', 'History', 'Science']].corr()

In [ ]:
mom_school_grades[['cumulative_gpa', 'English', 'Math', 'History', 'Science']].corr()

In [ ]:
mom_grad_level = pd.merge(mom_school_grades, educ_level, how = 'left', on = 'AID')
mom_grad_level['Value'].value_counts()
mom_grad_level.head()

In [ ]:
mom_grad_level[['graduate_college', 'graduate_hs', 'cumulative_gpa', 'Value']].corr()

In [ ]:
mom_grad_level.to_csv('mom_grad.csv')

Dad -- Relationship

In [ ]:
#significantly more kids who don't have dads (value 7) than who don't have moms
dad = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0001\21600-0001-Data.tsv", sep = '\t', usecols = ['AID', 'H1WP13', 'H1WP14'])
dad = dad.rename(columns = {'H1WP13':'dad_close', 'H1WP14':'caring_dad'})
dad = pd.merge(AID, dad, how = 'left', on = 'AID')
dad.isna().sum()
dad['dad_close'].value_counts() #drop 6(1), 7 (904)
dad['caring_dad'].value_counts() #drop 6(1), 7 (904)
dad = dad.drop(dad[(dad['dad_close'] == 7) | (dad['dad_close'] == 6)].index)
dad

In [ ]:
dc_code = pd.DataFrame(list(zip(code_rel, value_rel)), columns = ['dad_close', 'dad_close_code'])
cd_code = pd.DataFrame(list(zip(code_rel, value_rel)), columns = ['caring_dad', 'caring_dad_code'])

In [ ]:
dad = pd.merge(dad, dc_code, how = 'left', on = 'dad_close')
dad

In [ ]:
dad = pd.merge(dad, cd_code, how = 'left', on = 'caring_dad')
dad

In [ ]:
dad = pd.merge(dad, hs_gpa, on = 'AID', how = 'inner')
dad

In [ ]:
dad[['dad_close', 'caring_dad', 'cumulative_gpa']].corr()

In [ ]:
dad_grades = pd.merge(dad, grades, on = 'AID', how = 'inner')
dad_grades

In [ ]:
dad_grades[['dad_close', 'caring_dad','English', 'Math', 'History', 'Science']].corr()

In [ ]:
dad_grades[['cumulative_gpa', 'English', 'Math', 'History', 'Science']].corr()

In [ ]:
dad_educ_level = pd.merge(dad_grades, educ_level, how = 'left', on = 'AID')
dad_educ_level['Value'].value_counts()
dad_educ_level.head()

In [ ]:
dad_educ_level['gpa_range'].value_counts()

In [ ]:
dad_educ_level[['dad_close', 'caring_dad', 'cumulative_gpa', 'Value']].corr()

In [ ]:
dad_educ_level.to_csv('dad.csv')

Dad -- View on School -- Disappointed if they don't graduate college/hs

In [ ]:
dad_school = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0001\21600-0001-Data.tsv", sep = '\t', usecols = ['AID', 'H1WP15', 'H1WP16'])
dad_school = dad_school.rename(columns = {'H1WP15': 'graduate_college', 'H1WP16': 'graduate_hs'})
dad_school = pd.merge(AID, dad_school, on = 'AID', how = 'left')
dad_school.isna().sum()
dad_school['graduate_college'].value_counts() #drop 7 (904), 8(4), 6(1)
dad_school['graduate_hs'].value_counts() #drop 7(904), 8(3), 6(4)
dad_school = dad_school.drop(dad_school[dad_school['graduate_hs'] == 7].index)
dad_school.loc[(dad_school['graduate_hs'] == 8) | (dad_school['graduate_hs'] == 6)]
dad_school = dad_school.drop([1097, 1816, 3163])

In [ ]:
dad_school = pd.merge(dad_school, hs_gpa, on = 'AID', how = 'inner')
dad_school

In [ ]:
dad_school[['graduate_college', 'graduate_hs', 'cumulative_gpa']].corr()

In [ ]:
dad_school_grades = pd.merge(dad_school, grades, on = 'AID', how = 'inner')
dad_school_grades

In [ ]:
dad_school_grades[['graduate_college', 'graduate_hs','English', 'Math', 'History', 'Science']].corr()

In [ ]:
dad_school_grades[['cumulative_gpa','English', 'Math', 'History', 'Science']].corr()

In [ ]:
dad_grad_educ_level = pd.merge(dad_school_grades, educ_level, how = 'left', on = 'AID')
dad_grad_educ_level['Value'].value_counts()
dad_grad_educ_level.head()

In [ ]:
dad_grad_educ_level[['graduate_college', 'graduate_hs', 'cumulative_gpa', 'Value']].corr()

In [ ]:
dad_grad_educ_level.to_csv("dad_grad.csv")

Kids w/ no mom or dad figure

&emsp; No dad

In [ ]:
no_dad = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0001\21600-0001-Data.tsv", sep = '\t', usecols = ['AID', 'H1WP13', 'H1WP14'])
no_dad = no_dad.rename(columns = {'H1WP13':'dad_close', 'H1WP14':'caring_dad'})
no_dad = pd.merge(AID, no_dad, how = 'left', on = 'AID')
no_dad.isna().sum()
no_dad = no_dad.drop(no_dad[(no_dad['dad_close'] == 6)].index)
no_dad

In [ ]:
no_dad = no_dad.loc[no_dad['dad_close'] == 7]

In [ ]:
no_dad = pd.merge(no_dad, hs_gpa, on = 'AID', how = 'inner')

In [ ]:
no_dad = pd.merge(no_dad, grades, on = 'AID', how = 'inner')

In [ ]:
no_dad = pd.merge(no_dad, educ_level, how = 'inner', on = 'AID')

In [ ]:
no_dad

In [ ]:
no_dad['gpa_range'].value_counts()

In [ ]:
no_dad_school = pd.DataFrame(round(no_dad['Highest_educ_level'].value_counts(normalize = True), 4)).reset_index(drop = False)

&emsp; No Mom

In [ ]:
no_mom = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0001\21600-0001-Data.tsv", sep = '\t', usecols = ['AID','H1WP9', 'H1WP10'])
no_mom = no_mom.rename(columns = {'H1WP9': 'mom_close', 'H1WP10': 'caring_mom'})
no_mom = pd.merge(AID, no_mom, how = 'left', on = 'AID')
no_mom.isna().sum()
no_mom.loc[(no_mom['mom_close'] == 6) | (no_mom['mom_close'] == 8)]
no_mom = no_mom.drop([986, 1401, 1097, 3054])
no_mom = no_mom.loc[no_mom['mom_close'] == 7]
no_mom

In [ ]:
no_mom = pd.merge(no_mom, hs_gpa, on = 'AID', how = 'inner')
no_mom

In [ ]:
no_mom = pd.merge(no_mom, grades, on = 'AID', how = 'inner')
no_mom

In [ ]:
no_mom = pd.merge(no_mom, educ_level, how = 'inner', on = 'AID')
no_mom

In [ ]:
no_mom['gpa_range'].value_counts()

In [ ]:
no_dad_school = pd.DataFrame(round(no_mom['Highest_educ_level'].value_counts(normalize = True), 4)).reset_index(drop = False)

&emsp; No Mom nor Dad

In [ ]:
no_mom2 = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0001\21600-0001-Data.tsv", sep = '\t', usecols = ['AID','H1WP9', 'H1WP10'])
no_mom2 = no_mom2.rename(columns = {'H1WP9': 'mom_close', 'H1WP10': 'caring_mom'})
no_mom2 = pd.merge(AID, no_mom2, how = 'left', on = 'AID')
no_mom2.isna().sum()
no_mom2 = no_mom2.loc[no_mom2['mom_close'] == 7]
no_mom2

In [ ]:
no_dad2 = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0001\21600-0001-Data.tsv", sep = '\t', usecols = ['AID', 'H1WP13', 'H1WP14'])
no_dad2 = no_dad2.rename(columns = {'H1WP13':'dad_close', 'H1WP14':'caring_dad'})
no_dad2 = pd.merge(AID, no_dad2, how = 'left', on = 'AID')
no_dad2.isna().sum()
no_dad2 = no_dad2.loc[no_dad2['dad_close'] == 7]
no_dad2

In [ ]:
no_parents = pd.merge(no_mom2, no_dad2, on = 'AID', how = 'inner')

In [ ]:
no_parents = pd.merge(no_parents, hs_gpa, on = 'AID', how = 'inner')
len(no_parents)

In [ ]:
no_parents = pd.merge(no_parents, grades, on = 'AID', how = 'inner')
no_parents

In [ ]:
no_parents = pd.merge(no_parents, educ_level, how = 'inner', on = 'AID')
no_parents

In [ ]:
no_parents['gpa_range'].value_counts()

In [ ]:
pd.DataFrame(round(no_parents['Highest_educ_level'].value_counts(normalize = True),4)).reset_index(drop = False)

&emsp; Single Mom

In [ ]:
mom2 = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0001\21600-0001-Data.tsv", sep = '\t', usecols = ['AID','H1WP9', 'H1WP10'])
mom2 = mom2.rename(columns = {'H1WP9': 'mom_close', 'H1WP10': 'caring_mom'})
mom2 = pd.merge(AID, mom2, how = 'left', on = 'AID')
mom2.isna().sum()
mom2 = mom2.loc[mom2['mom_close'] != 7]
mom2

In [ ]:
single_mom = pd.merge(mom2, no_dad, how = 'inner', on = 'AID')
single_mom['mom_close'].value_counts()
single_mom = single_mom.drop(single_mom[single_mom['mom_close'] == 8].index)

In [ ]:
single_mom

In [ ]:
single_mom['mom_close'].value_counts()

In [ ]:
single_mom['gpa_range'].value_counts()

In [ ]:
single_mom_school = pd.DataFrame(round(single_mom['Highest_educ_level'].value_counts(normalize = True),4)).reset_index(drop = False)
single_mom_school

In [ ]:
single_mom[['mom_close', 'caring_mom', 'cumulative_gpa', 'Value']].corr()

&emsp; Single Dad

In [ ]:
dad2 = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0001\21600-0001-Data.tsv", sep = '\t', usecols = ['AID', 'H1WP13', 'H1WP14'])
dad2 = dad2.rename(columns = {'H1WP13':'dad_close', 'H1WP14':'caring_dad'})
dad2 = pd.merge(AID, dad2, how = 'left', on = 'AID')
dad2.isna().sum()
dad2 = dad2.loc[dad2['dad_close'] != 7]
dad2

In [ ]:
single_dad = pd.merge(dad2, no_mom, how = 'inner', on = 'AID')
single_dad

In [ ]:
single_dad['dad_close'].value_counts()

In [ ]:
single_dad['gpa_range'].value_counts()

In [ ]:
pd.DataFrame(round(single_dad['Highest_educ_level'].value_counts(normalize = True), 4)).reset_index(drop = False)

In [ ]:
single_dad[['dad_close', 'caring_dad', 'cumulative_gpa', 'Value']].corr()

&emsp; Both Parents

In [ ]:
mom3 = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0001\21600-0001-Data.tsv", sep = '\t', usecols = ['AID','H1WP9', 'H1WP10'])
mom3 = mom3.rename(columns = {'H1WP9': 'mom_close', 'H1WP10': 'caring_mom'})
mom3 = pd.merge(AID, mom3, how = 'left', on = 'AID')
mom3.isna().sum()
mom3 = mom3.loc[mom3['mom_close'] != 7]
mom3

In [ ]:
mom3['mom_close'].value_counts()

In [ ]:
dad3 = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0001\21600-0001-Data.tsv", sep = '\t', usecols = ['AID', 'H1WP13', 'H1WP14'])
dad3 = dad3.rename(columns = {'H1WP13':'dad_close', 'H1WP14':'caring_dad'})
dad3 = pd.merge(AID, dad3, how = 'left', on = 'AID')
dad3.isna().sum()
dad3['dad_close'].value_counts() #drop 6(1), 7 (904)
dad3['caring_dad'].value_counts() #drop 6(1), 7 (904)
dad3 = dad3.drop(dad3[(dad3['dad_close'] == 7) | (dad3['dad_close'] == 6)].index)
dad3

In [ ]:
dad3['dad_close'].value_counts()
dad3['caring_dad'].value_counts()

In [ ]:
dad3 = pd.merge(dad3, dc_code, how = 'left', on = 'dad_close')
dad3

In [ ]:
dad3 = pd.merge(dad3, cd_code, how = 'left', on = 'caring_dad')
dad3

In [ ]:
mom3 = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0001\21600-0001-Data.tsv", sep = '\t', usecols = ['AID','H1WP9', 'H1WP10'])
mom3 = mom3.rename(columns = {'H1WP9': 'mom_close', 'H1WP10': 'caring_mom'})
mom3 = pd.merge(AID, mom3, how = 'left', on = 'AID')
mom3.isna().sum()
mom3['mom_close'].value_counts() #drop 8 and 6 (2 each) and 7 (148)
mom3['caring_mom'].value_counts() #drop 8 and 6 if overlap (1 each) and 7 (148)
mom3.loc[(mom3['mom_close'] == 6) | (mom3['mom_close'] == 8)]
mom3 = mom3.drop(mom3[mom3['mom_close'] == 7].index)
mom3 = mom3.drop([986, 1097, 1401, 3054])
mom3

In [ ]:
mom3['mom_close'].value_counts()
mom3['caring_mom'].value_counts()

In [ ]:
mom3 = pd.merge(mom3, mc_code, how = 'left', on = 'mom_close')
mom3

In [ ]:
mom3 = pd.merge(mom3, cm_code, how = 'left', on = 'caring_mom')
mom3

In [ ]:
both_parents = pd.merge(mom3, dad3, on = 'AID', how = 'inner')
both_parents = pd.merge(both_parents, hs_gpa, on = 'AID', how = 'inner')
both_parents = pd.merge(both_parents, educ_level, on = 'AID', how = 'inner')
both_parents = pd.merge(both_parents, grades, on = 'AID', how = 'inner')
both_parents

In [ ]:
both_parents['gpa_range'].value_counts()

In [ ]:
both_parents['Highest_educ_level'].value_counts()

In [ ]:
both_parents[['mom_close', 'caring_mom', 'dad_close', 'caring_dad', 'cumulative_gpa', 'Value']].corr()

CSV Files

In [ ]:
both_parents.to_csv('both parents.csv')

In [ ]:
no_parents.to_csv('no parents.csv')

In [ ]:
single_mom.to_csv('single mom.csv')

In [ ]:
single_dad.to_csv('single dad.csv')